# ОИАД. Лабораторная работа №3

## 1. Подготовка данных
1. проверить наличие пропусков и выбросов
2. привести категориальные признаки к числовым
3. вычислить парные корреляции признаков


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df = pd.read_csv('insurance.csv')

In [5]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [7]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [8]:
df['sex'].unique()

array(['female', 'male'], dtype=object)

In [9]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [10]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [11]:
df.drop_duplicates(inplace=True)

In [13]:
full_df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], prefix=['sex_onehot_', 'smoker_onehot_', 'region_onehot_'], dtype='int64')
train_df, test_df = train_test_split(full_df, test_size=0.3)

train_X = train_df.drop(['charges'], axis=1)
train_Y = train_df.charges

test_X = test_df.drop(['charges'], axis=1)
test_Y = test_df.charges

print("Train X shape: ", train_X.shape)
print("Test X shape: ", test_X.shape)

train_X.head()

Train X shape:  (935, 11)
Test X shape:  (402, 11)


,age,bmi,children,sex_onehot__female,sex_onehot__male,smoker_onehot__no,smoker_onehot__yes,region_onehot__northeast,region_onehot__northwest,region_onehot__southeast,region_onehot__southwest
1094,50,33.70,4,1,0,1,0,0,0,0,1
1245,28,24.30,5,0,1,1,0,0,0,0,1
1289,44,34.32,1,0,1,1,0,0,0,1,0
770,61,36.10,3,0,1,1,0,0,0,0,1
272,41,37.05,2,0,1,1,0,0,1,0,0


In [15]:
train_Y.head()

1094    11299.34300
1245     5615.36900
1289     7147.47280
770     27941.28758
272      7265.70250
Name: charges, dtype: float64

In [16]:
correlation_df = train_X.corr()
print(correlation_df)

                               age       bmi  children  sex_onehot__female  \
age                       1.000000  0.108140  0.026485            0.031508   
bmi                       0.108140  1.000000  0.006118           -0.041886   
children                  0.026485  0.006118  1.000000           -0.036750   
sex_onehot__female        0.031508 -0.041886 -0.036750            1.000000   
sex_onehot__male         -0.031508  0.041886  0.036750           -1.000000   
smoker_onehot__no         0.050979 -0.032787 -0.009942            0.060216   
smoker_onehot__yes       -0.050979  0.032787  0.009942           -0.060216   
region_onehot__northeast  0.023247 -0.126956 -0.032894            0.016281   
region_onehot__northwest -0.001199 -0.146413  0.034221            0.004008   
region_onehot__southeast -0.020090  0.299995 -0.021830           -0.004343   
region_onehot__southwest -0.001107 -0.037359  0.020860           -0.015587   

                          sex_onehot__male  smoker_onehot__no  

## 2. Многомерная линейная регрессия
Построить модель линейной регрессии и подобрать параметры:
1. аналитически (реализовать самому)
2. численно, с помощью методов градиентного спуска (реализовать самому)

In [82]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc_y = StandardScaler()
X_normalized = sc.fit_transform(train_X)
Y_normalized = pd.Series(sc_y.fit_transform(pd.DataFrame(train_Y)).flatten())
#X_normalized = (train_X-train_X.min())/(train_X.max()-train_X.min())

In [83]:
w_hat = np.linalg.inv(X_normalized.T @ X_normalized) @ X_normalized.T @ Y_normalized

In [84]:
W = pd.Series(w_hat.flatten(), index=train_X.columns)
W.shape

(11,)

In [85]:
y_hat = X_normalized.dot(W)
y_hat = pd.Series(sc_y.inverse_transform(y_hat.reshape(-1, 1)).flatten())
y_hat.head()

0    13503.659901
1     6104.274702
2    10542.166737
3    16588.745587
4    11736.922799
dtype: float64

In [88]:
eps = 1e-6
lambda_ = 1e-5
w_cap = np.zeros(X_normalized.shape[1])
max_iter = 100000

for i in range(max_iter):
    grad = 2 * X_normalized.T @ (X_normalized @ w_cap - Y_normalized)
    grad_norm = np.linalg.norm(grad)
    
    if grad_norm < eps:
        print(f"Сходимость на итерации {i}")
        break
    
    w_cap = w_cap - lambda_ * grad
    
    if i % 1000 == 0:
        print(f"Iteration {i}, grad_norm: {grad_norm:.6f}")

print("Готово!")

Iteration 0, grad_norm: 2203.515078
Iteration 1000, grad_norm: 0.000038
Сходимость на итерации 1265
Готово!


In [93]:
y_cap = X_normalized.dot(w_cap)
y_cap = pd.Series(sc_y.inverse_transform(y_cap.reshape(-1, 1)).flatten())
y_cap.head()

0    13328.273620
1     5342.432137
2    10545.616483
3    16362.137529
4    11738.020382
dtype: float64

## 3. Добавление регуляризации
Модифицировать линейную модель путем добавления регуляризационного слагаемого. Найти оптимальные веса:
1. аналитически
2. численно

### Ridge Regression

In [129]:
alpha = 1
ones = np.eye(X_normalized.shape[1])
w_hat_ridge = np.linalg.inv(X_normalized.T @ X_normalized + alpha * ones) @ X_normalized.T @ Y_normalized

In [130]:
W_ridge = pd.Series(w_hat_ridge.flatten(), index=train_X.columns)
y_hat_ridge = X_normalized.dot(W_ridge)
y_hat_ridge = pd.Series(sc_y.inverse_transform(y_hat_ridge.reshape(-1, 1)).flatten())
y_hat_ridge.head()

0    13325.565477
1     5348.683335
2    10548.013081
3    16356.745464
4    11737.275398
dtype: float64

In [118]:
eps = 1e-7
lambda_ = 1e-5
w_cap_ridge = np.zeros(X_normalized.shape[1])
max_iter = 100000

for i in range(max_iter):
    grad_ridge = 2 * X_normalized.T @ (X_normalized @ w_cap_ridge - Y_normalized) + 2 * alpha * w_cap_ridge
    grad_norm_ridge = np.linalg.norm(grad_ridge)
    
    if grad_norm_ridge < eps:
        print(f"Сходимость на итерации {i}")
        break
    
    w_cap_ridge = w_cap_ridge - lambda_ * grad_ridge
    
    if i % 1000 == 0:
        print(f"Iteration {i}, grad_norm: {grad_norm_ridge:.6f}")

print("Готово!")

Iteration 0, grad_norm: 2203.515078
Iteration 1000, grad_norm: 0.000037
Сходимость на итерации 1430
Готово!


In [119]:
y_cap_ridge = X_normalized.dot(w_cap_ridge)
y_cap_ridge = pd.Series(sc_y.inverse_transform(y_cap_ridge.reshape(-1, 1)).flatten())
y_cap_ridge.head()

0    13325.565477
1     5348.683335
2    10548.013081
3    16356.745464
4    11737.275397
dtype: float64

### Lasso Regression

In [106]:
beta = 1
eps = 1e-6
lambda_ = 1e-5
w_cap_lasso = np.zeros(X_normalized.shape[1])
max_iter = 50000

for i in range(max_iter):
    grad_lasso = 2 * X_normalized.T @ (X_normalized @ w_cap_lasso - Y_normalized) + beta * np.sign(w_cap_lasso)
    grad_norm_lasso = np.linalg.norm(grad_lasso)
    
    if grad_norm_lasso < eps:
        print(f"Сходимость на итерации {i}")
        break
    
    w_cap_lasso = w_cap_lasso - lambda_ * grad_lasso
    
    if i % 1000 == 0:
        print(f"Iteration {i}, grad_norm: {grad_norm_lasso:.6f}")

print("Готово!")

Iteration 0, grad_norm: 2203.515078
Iteration 1000, grad_norm: 0.023178
Iteration 2000, grad_norm: 0.023178
Iteration 3000, grad_norm: 0.023178
Iteration 4000, grad_norm: 0.023178
Iteration 5000, grad_norm: 0.023178
Iteration 6000, grad_norm: 0.023178
Iteration 7000, grad_norm: 0.023178
Iteration 8000, grad_norm: 0.023178
Iteration 9000, grad_norm: 0.023178
Iteration 10000, grad_norm: 0.023178
Iteration 11000, grad_norm: 0.023178
Iteration 12000, grad_norm: 0.023178
Iteration 13000, grad_norm: 0.023178
Iteration 14000, grad_norm: 0.023178
Iteration 15000, grad_norm: 0.023178
Iteration 16000, grad_norm: 0.023178
Iteration 17000, grad_norm: 0.023178
Iteration 18000, grad_norm: 0.023178
Iteration 19000, grad_norm: 0.023178
Iteration 20000, grad_norm: 0.023178
Iteration 21000, grad_norm: 0.023178
Iteration 22000, grad_norm: 0.023178
Iteration 23000, grad_norm: 0.023178
Iteration 24000, grad_norm: 0.023178
Iteration 25000, grad_norm: 0.023178
Iteration 26000, grad_norm: 0.023178
Iteration 2

In [107]:
y_cap_lasso = X_normalized.dot(w_cap_lasso)
y_cap_lasso = pd.Series(sc_y.inverse_transform(y_cap_lasso.reshape(-1, 1)).flatten())
y_cap_lasso.head()

0    13325.259567
1     5345.344995
2    10552.005058
3    16344.073296
4    11706.688442
dtype: float64

## 4. Оценка обобщающей способности
Сравнить между собой модели на тестовых данных по среднему квадрату ошибки:
1. константную - прогноз средним значением
2. из пункта 2
3. из пункта 3

In [120]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [137]:
X_normalized_test = sc.transform(test_X)
Y_normalized_test = pd.Series(sc_y.transform(pd.DataFrame(test_Y)).flatten())

### LenearRegression (Sci-kit Learn)

In [139]:
reg = LinearRegression().fit(X_normalized, Y_normalized)
y_preds_norm = reg.predict(X_normalized_test)
y_preds = pd.Series(sc_y.inverse_transform(y_preds_norm.reshape(-1, 1)).flatten())

approx_score = mse(y_preds, test_Y)
print(f"MSE for sklearn LR approximation: {approx_score}")

MSE for sklearn LR approximation: 32057862.035308056


### Приближение средним

In [108]:
y_mean = test_Y.mean()
y_mean_series = pd.Series([y_mean for _ in range(test_Y.shape[0])]) 

In [109]:
mean_approx_score = mse(y_mean_series, test_Y)
print(f"MSE for mean approximation: {mean_approx_score}")

MSE for mean approximation: 132406844.84243308


### Приближение линейной регрессией

In [111]:
y_hat = X_normalized_test.dot(W)
y_hat = pd.Series(sc_y.inverse_transform(y_hat.reshape(-1, 1)).flatten())
y_hat_approx_score = mse(y_hat, test_Y)
print(f"MSE for analitical LR approximation: {y_hat_approx_score}")

MSE for analitical LR approximation: 32198698.057940487


In [132]:
y_cap = X_normalized_test.dot(w_cap)
y_cap = pd.Series(sc_y.inverse_transform(y_cap.reshape(-1, 1)).flatten())
y_cap_approx_score = mse(y_cap, test_Y)
print(f"MSE for numerical LR approximation: {y_cap_approx_score}")

MSE for numerical LR approximation: 32057862.03878147


### Приближение Линейной регрессией с регуляризацией

In [134]:
y_hat_ridge = X_normalized_test.dot(W_ridge)
y_hat_ridge = pd.Series(sc_y.inverse_transform(y_hat_ridge.reshape(-1, 1)).flatten())
y_hat_ridge_approx_score = mse(y_hat_ridge, test_Y)
print(f"MSE for analitical LR /w L1 reg. approximation: {y_hat_ridge_approx_score}")

MSE for analitical LR /w L1 reg. approximation: 32057001.8759964


In [135]:
y_cap_ridge = X_normalized_test.dot(w_cap_ridge)
y_cap_ridge = pd.Series(sc_y.inverse_transform(y_cap_ridge.reshape(-1, 1)).flatten())
y_cap_ridge_approx_score = mse(y_cap_ridge, test_Y)
print(f"MSE for numerical LR /w L1 reg. approximation: {y_cap_ridge_approx_score}")

MSE for numerical LR /w L1 reg. approximation: 32057001.876334637


In [136]:
y_cap_lasso = X_normalized_test.dot(w_cap_lasso)
y_cap_lasso = pd.Series(sc_y.inverse_transform(y_cap_lasso.reshape(-1, 1)).flatten())
y_cap_lasso_approx_score = mse(y_cap_lasso, test_Y)
print(f"MSE for numerical LR /w L2 reg. approximation: {y_cap_lasso_approx_score}")

MSE for numerical LR /w L2 reg. approximation: 32052051.777075537
